### QPDK Coupled Resonator — Eigenmode Simulation

This notebook builds a compact coupled resonator from QPDK cells, converts etch
layers to conductor geometry, and runs a Palace eigenmode simulation to find
resonant frequencies and quality factors.

In [ ]:
import gdsfactory as gf

from qpdk import PDK, cells
from qpdk.cells.airbridge import cpw_with_airbridges
from qpdk.tech import LAYER, route_bundle_sbend_cpw

PDK.activate()


@gf.cell
def resonator_compact(coupling_gap: float = 20.0) -> gf.Component:
    """Compact coupled resonator with S-bend CPW routes."""
    c = gf.Component()

    res = c << cells.resonator_coupled(
        coupling_straight_length=300, coupling_gap=coupling_gap
    )
    res.movex(-res.size_info.width / 4)

    left = c << cells.straight()
    right = c << cells.straight()

    w = res.size_info.width + 100
    left.move((-w, 0))
    right.move((w, 0))

    route_bundle_sbend_cpw(
        c,
        [left["o2"], right["o1"]],
        [res["coupling_o1"], res["coupling_o2"]],
        cross_section=cpw_with_airbridges(
            airbridge_spacing=250.0, airbridge_padding=20.0
        ),
    )

    c.kdb_cell.shapes(LAYER.SIM_AREA).insert(c.bbox().enlarged(0, 100))

    c.add_port(name="o1", port=left["o1"])
    c.add_port(name="o2", port=right["o2"])
    return c


component = resonator_compact(coupling_gap=15.0)
_c = component.copy()
_c.draw_ports()
_c

### Convert QPDK etch layers to conductor geometry

In [ ]:
from gsim.common.stack.qpdk_utils import create_etched_component

etched = create_etched_component(component)
etched

### Configure simulation

In [ ]:
from gsim.palace import EigenmodeSim

sim = EigenmodeSim()
sim.set_geometry(etched)
sim.set_stack(substrate_thickness=500, air_above=500)
sim.add_cpw_port("o1", layer="CONDUCTOR", s_width=10.0, gap_width=6.0, length=5.0)
sim.add_cpw_port("o2", layer="CONDUCTOR", s_width=10.0, gap_width=6.0, length=5.0)
sim.set_eigenmode(target=7.5e9, num_modes=3)

In [ ]:
### Mesh and run

In [ ]:
sim.set_output_dir("./sim_qpdk_resonator")
sim.mesh(preset="default")
sim.plot_mesh()

In [ ]:
results = sim.run()

if results.ok:
    print("Eigenvalues")
    print(f"{'Freq (GHz)':<16} {'Q':<16}")
    for ev in results.eigenvalues:
        print(f"{ev.freq:<16.4f} {ev.quality_factor:<16.4f}")
else:
    print("Simulation failed:", results.error_msg)